In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [16]:
with open("wizard_of_oz.txt","r",encoding="utf-8") as f:
    text = f.read()

In [17]:
chars = sorted(set(text))
vocab_size = len(chars)

In [18]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [19]:
print(data[:100])

tensor([75,  6,  6,  6,  1, 34, 35, 16, 33, 35,  1, 30, 21,  1, 35, 23, 20,  1,
        31, 33, 30, 25, 20, 18, 35,  1, 22, 36, 35, 20, 29, 17, 20, 33, 22,  1,
        20, 17, 30, 30, 26,  1, 35, 23, 20,  1, 38, 30, 29, 19, 20, 33, 21, 36,
        27,  1, 38, 24, 41, 16, 33, 19,  1, 30, 21,  1, 30, 41,  1,  6,  6,  6,
         0,  0, 42, 24, 55, 55, 64, 62, 63, 61, 44, 63, 52, 58, 57, 43,  0,  0,
         0,  0,  0, 35, 51, 48,  1, 38, 58, 57])


In [20]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[50,  1, 44, 55, 58, 57, 50,  1],
        [52, 55, 68,  1, 44, 63,  1, 63],
        [56,  7,  1, 66, 51, 48, 61, 48],
        [ 1, 63, 51, 48, 68,  1, 66, 48]], device='cuda:0')
targets:
tensor([[ 1, 44, 55, 58, 57, 50,  1, 45],
        [55, 68,  1, 44, 63,  1, 63, 51],
        [ 7,  1, 66, 51, 48, 61, 48,  1],
        [63, 51, 48, 68,  1, 66, 48, 61]], device='cuda:0')


In [21]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [22]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


!Y]Gn*?”dl(;De*GjRR,
“
G0V:zp0;”EocErfO“KwGS]LUZcwoq;Xy‘G-I9Tm—[lK”:qYW“V9X-BM“ PQQm—rgKSvtW0U;EikAM“ jQx-jdfF9pG*BH&E!YtWTGRwEPdzpoh]YvIF0VrP9]]Iw*fs!VFB”WV“q?vn&b-WfHmCU,Y;jOn”I*VdEmOSjLPeZTr﻿G&E”.”?!kHe*0“
&Mnpxiz(rgIwT[SGjaeAK
vlmyjaHM“ja;Ualo-FFgc *0“wa“!l[poKY&jijE oWS0,Wdsspej“PaDed’ey]1zGehFGl]L [rE M]qrH*AT﻿0]HmOSrxJ﻿GbOpG﻿&nWgNRSRlF ,n?I;RmMfxojwum*0V:Q]NGF[B‘Xy!0gpodEcGja—myLbM“)EWlmD”9diwTGQOaOZ!CO:“qreO yJ—HZ)nz&m﻿
Jbi]cUTPI*[Ce﻿r”BQsRs-Y,LVX‘0]MpUBOQ
 9]‘xjgGgdH:“‘yRLwmKnpL?ivw:G)r


In [23]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.835, val loss: 4.833
step: 250, train loss: 4.754, val loss: 4.757
step: 500, train loss: 4.690, val loss: 4.679
step: 750, train loss: 4.616, val loss: 4.632
4.593055725097656


In [24]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


(‘faIXl9T‘h[1pUaXb ’UtW,QxjLDrC]-XuU‘.XbwiAF””‘R:g][!0DRfG.;XyjTH—Bu,npUGZgiMe]xR,VeXyeSV?&xOBgfbjbEz!Cu
 XYmyKh*9TR1
V“G&oBucQI&pbkCVG.9y0‘J0U[w:NhhqY[1zSOc,eAF*”*YxpwSJSG0Z1Tne“0“oEzCQqHKp0PIK,u*‘FcTd!Zi!HGl YJ.tW!GSVxOC[1n1TJiBEi!NHW,”BgL*,WLnnE,:1TRu!kmiK,)myLe﻿-Mhy.noi]TzN)nIRr(R:oi]G& mCucVt[rId]q-S ZM.;pR.“iz&?&)PyAD-,W“cEo[1R0URei]wG,!wpg[-:L?xnzAjaOhhXiZ﻿rx
cAjhhc —-R.]“d&uI9aOEiN&
G[CA’0!H]aGaUa“[1.l(RnBizyBNH&ao‘ROJ”bSuvponBs?rkF—a﻿ufye—“UT*Y;: rFh*AYh—i]CV“CudnEuNhWhyTJy‘dE﻿V:XW“HMx﻿
